# Stuff I Like About The Julia Programming Language

`git_club 2020-06-19` - Hannes

> Julia's unofficial tagline is "Looks like Python, feels like Lisp, runs like Fortran."
> I've been learning Julia for slightly less than a year now, and I'd like to share some of the things I've enjoyed about it.
> I'll give an overview of the language's main features with code examples and discuss whether it really is the future scientific computing.

* [Static notebook on nbviewer](https://nbviewer.jupyter.org/github/Hasnep/stuff-i-like-about-julia/blob/master/stuff-i-like-about-julia.ipynb)
* [Interactive notebook on Binder](https://mybinder.org/v2/gh/Hasnep/stuff-i-like-about-julia/master?filepath=stuff-i-like-about-julia.ipynb)

## Expectations

- I'm not going to:
    - Convince you to switch to Julia
    - Teach you everything about Julia
    - Convince you that Julia is better than Python
- I'll try to:
    - Use Python as a comparison to give you some context
    - Show you some things I like about Julia   

## What is Julia?

Technically:

> Julia is a high level, JIT compiled, dynamic language designed with multiple dispatch, automatic differentiation and metaprogramming.

In practice:

> Julia finds a balance between being fast and easy to use with lots of features you'll miss when you use another language.

## History

- Julia was started in 2009 by Jeff Bezanson, Stefan Karpinski, Viral B. Shah, and Alan Edelman
- Announced in 2012
- Version 1.0 was released in 2018
- Now on version 1.4, heading for 1.5 soon

## It's fast

Julia was designed to solve the "two language problem", where researchers use a slower, high-level language for research and then have to use a slower, low-level language once they hit a bottleneck or for production.

Julia is both languages at the same time!

![Benchmarks of different languages relative to C](https://julialang.org/assets/benchmarks/benchmarks.svg)

> Source: https://julialang.org/benchmarks/

Some benchmarks (from the Julia website) of different languages relative to C.
These benchmarks try to compare implementations of the same algorithm.

Julia is comparable to compiled languages like Rust, Go, Fortran, etc. and is sometimes faster than C.
Python is sometimes 100x slower than C.
R is a bit slower than that.


### How?

- Compiled languages like C or Rust compile all the code before you run
- Interpreted languages like Python or R don't compile
- Julia uses a _Just-In-Time_ (JIT) compiler which compiles a function the first time it is called.

### Why I don't think it matters that much

* My time is more valuable than the computer's time
* JIT compilation takes a while

### Easy to use

Writing a sum function from scratch, first in Python:

```python
def my_sum(array):
    """
    Sum a list.
    """
    total = 0
    for x in array:
        total += x
    return total
```

Then in Julia:

In [ ]:
"""
Sum an array.
"""
function my_sum(array)
    total = 0
    for x in array
        total += x
    end
    return total
end

This syntax will be familliar if you've used Python.

In [ ]:
my_sum(1:10)

- C: ~10ms
- Python: ~500ms
- Julia: ~10ms

> Source: [An Introduction to Julia (Beginner Level) | SciPy 2018 Tutorial | Jane Herriman, Sacha Verweij](https://www.youtube.com/watch?v=b5xvVyzUnXI)

There are two ways to write functions in Julia:

In [ ]:
"""
Calculate the nth Fibonacci number.
"""
function long_fib(n)
    if n < 2
        return n
    else
        return long_fib(n - 1) + long_fib(n - 2)
    end
end

In [ ]:
long_fib(20)

In [ ]:
short_fib(n) = n < 2 ? n : short_fib(n - 1) + short_fib(n - 2)

In [ ]:
short_fib(20)

## Broadcasting

In Python, most functions accept one element.

```python
>>> import math
>>> math.sin([1, 2, 3])
```

will give an error:

```
TypeError: must be real number, not list
```

You can use a list comprehension:

```python
>>> [math.sin(x) for x in [1, 2, 3]]
```

or a map:

```python
>>> map(math.sin, [1, 2, 3])
```

```
[0.8414709848078965, 0.9092974268256817, 0.1411200080598672]
```

In R, most functions are vectorised

```R
> sin(c(1, 2, 3))
```

```
[1] 0.8414710 0.9092974 0.1411200
```

In Julia

```julia
julia> sin([1, 2, 3])
```

gives an error

```
ERROR: MethodError: no method matching sin(::Array{Int64,1})
```

Using the broadcast operator, the funciton is applied elementwise!

In [ ]:
sin.([1, 2, 3])

This is powerful, but sometimes tricky syntax.
For example, adding a dot makes the length function broadcast over the array:

In [ ]:
length(split("How many words are in this sentence?"))

In [ ]:
length.(split("How many characters are each of these words?"))

Broadcasting even works for user functions

In [ ]:
short_fib.(1:10)

And for operators

What are the first 10 square numbers?

In [ ]:
(1:10) .^ 2

### Julia is (mostly) written in Julia

If you can read Julia code, you can also read Julia's source code to understand what it does.

I looked at the most recent PR as an example:

In [ ]:
tensor(A::AbstractArray, B::AbstractArray) = [a * b for a in A, b in B]
const ⊗ = tensor

In [ ]:
[1, 2] ⊗ [3, 4, 5]

Python's numpy is fast because it's mostly written in C/C++ (51.4%), but if you want to do something that numpy can't do, you need to either use C or write slower Python code.

## Multiple dispatch

An example, based on [Measurements.jl](https://github.com/JuliaPhysics/Measurements.jl/).

In [ ]:
"""
A number with some error.
"""
struct Uncertain <: Real
    val::Real
    err::Real
end

Define the standard gravity on earth.

In [ ]:
g = Uncertain(9.8, 0.1)
print(g)

Wouldn't it be nicer to show an uncertain number with the plus/minus symbol?

In [ ]:
# Write a show function that dispatches on the Uncertain type
Base.show(io::IO, x::Uncertain) = print(io, "$(x.val) ∓ $(x.err)")
print(g)

Let's define an operator

In [ ]:
∓(a, b) = Uncertain(a, b)
my_height = 190 ∓ 1

How do you add two uncertain measurements?

$$
Q = a + b \\
{\delta Q} = \sqrt{(\delta a)^2 + (\delta b)^2}
$$

In [ ]:
Base.:+(a::Uncertain, b::Uncertain) = (a.val + b.val) ∓ sqrt(a.err^2 + b.err^2)

In [ ]:
my_brothers_height = 160 ∓ 1
my_height + my_brothers_height

Similar for subtraction.

In [ ]:
Base.:-(a::Uncertain, b::Uncertain) = Uncertain(a.val - b.val, sqrt(a.err^2 + b.err^2))

In [ ]:
my_height - my_brothers_height

Slightly more complicated for multiplcation and division.

In [ ]:
function Base.:*(a::Uncertain, b::Uncertain) 
    total_value = a.val * b.val
    total_error = total_value * sqrt((a.err / a.val)^2 + (b.err / b.val)^2)
    return Uncertain(total_value, total_error)
end

function Base.:/(a::Uncertain, b::Uncertain) 
    total_value = a.val / b.val
    total_error = total_value * sqrt((a.err / a.val)^2 + (b.err / b.val)^2)
    return Uncertain(total_value, total_error)
end

In [ ]:
println(my_height * my_brothers_height)
println(my_height / my_brothers_height)

Finally powers, again the exact formula is not important.

In [ ]:
Base.:^(a::Uncertain, b::Real) = (a.val^b) ∓ (abs(b) * a.val^(b - 1) * a.err)

In [ ]:
Base.promote_rule(::Type{Uncertain}, ::Type{T}) where T <: Real = Uncertain
Base.convert(::Type{Uncertain}, x::Real) = Uncertain(x, 0)
Base.convert(::Type{Uncertain}, x::Uncertain) = x

Solve for t:
$$
t = \frac{\sqrt{2 a s + u^2} - u}{a}
$$

In [ ]:
t = ((2 * g * (my_height - my_brothers_height))^0.5) / g

Let's use the actual Measurements.jl package

In [ ]:
using Measurements: Measurement, ±

In [ ]:
g = 9.8 ± 0.1
my_height = 190 ± 1
my_brothers_height = 160 ± 1

In [ ]:
t = ((2 * g * (my_height - my_brothers_height))^0.5) / g

In [ ]:
using DifferentialEquations
f(s,p,t) = (-9.8 ± 0.1) * t
s₀ = 190 ± 1
tspan = (0.0, 3.0)
problem = ODEProblem(f, s₀, tspan)
solution = solve(problem, Tsit5(),  saveat = 0.1)

In [ ]:
using Plots
plot(solution.t,solution.u,
title = "Solution to the ODE",
     xaxis = "Time (t) in seconds",
     yaxis = "Displacement s(t) in metres") 

## PyCall

In [ ]:
using PyCall
math = pyimport("math")
math.sin(math.pi / 4)

In [ ]:
py"""
def pyfib(n, fun):
    print("🐍")
    if n < 2:
        return n
    else:
        return fun(n - 1, pyfib) + fun(n - 2, pyfib)
"""

function jlfib(n, fun)
    print("🐈")
    if n < 2
        return n
    else
        return fun(n - 1, jlfib) + fun(n - 2, jlfib)
    end
end

## Disadvantages

- Super slow to load
  - That JIT compiler means the first time a function is run it's super slow as it compiles
    - It's instant the seocnd time you run it (in the same session), but for a function that only runs once, it's not great.
    - The way people talk about this is the "time to first plot problem", i.e. how long does it take to start the language's REPL, load some data packages, load some data and then plot it.
	  - For python/R it's < 5 seconds int total
	  - but for Julia you can wait like 10 seconds for a plotting package like `Gadfly.jl` to load, then another 5-10 seconds to produce the first plot. After that is done, plotting the second time is very quick obvs.
  - Being worked on though. There was recently a 15% speed up and there should be another 20% speed up soon, so < 10 seconds.
  - The current advice is to leave the REPL/Jupyter notebook open while you work on it. You can use `Revise.jl` to automatically reload your package if you're developing a package.
- Some (i.e. a lot of) packages are just not there yet in terms of maturity
- Julia's integers are machine integers, so they overflow
- Some more general programming things are harder to do in Julia, e.g. GUIs, , serving web stuff (although this is aparently a lot better with the new `Genie.jl` framework)
- Currently the only way to compile a binary is to include all of Julia in each binary.
  - This is being worked on 

## Julia in action

- Aviva, the UK’s largest insurer, uses the language for calculating complex risk models
- The Boston Public School District used Julia to optimize school bus routes, saving over $5 million in tax funds
- NASA ran it on one of the top five supercomputers in the world, cataloging every visible astronomical light source in the observable universe for the first time in a mere 15 minutes

> Source: [Julia: A Solution to the Two-Language Programming Problem](https://thebottomline.as.ucsb.edu/2018/10/julia-a-solution-to-the-two-language-programming-problem)